In [ ]:
# import libraries
import nltk
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

nltk.download('punkt')
nltk.download('stopwords')
import sqlite3

In [ ]:
# define DB class that interacts with the database

class Db:
    def __init__(self):
        self.conn_string = 'data/university_chatbot.db'
        self.init_db()
    
    def is_already_initialized(self):
        return self.get_one("SELECT name FROM sqlite_master WHERE type='table' AND name='students'")

    def init_db(self):

        if self.is_already_initialized():
            return
        
        # Create tables
        # Students Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS students (
                matriculation_number INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                surname TEXT NOT NULL,
                address TEXT
            )
            ''')

        # Courses Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS courses (
                course_id INTEGER PRIMARY KEY,
                course_name TEXT NOT NULL,
                instructor TEXT NOT NULL
            )
            ''')

        # Registrations Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS registrations (
                registration_id INTEGER PRIMARY KEY,
                matriculation_number INTEGER NOT NULL,
                course_id INTEGER NOT NULL,
                FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
                FOREIGN KEY (course_id) REFERENCES courses (course_id)
            )
            ''')

        # Exam Results Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS exam_results (
                result_id INTEGER PRIMARY KEY,
                matriculation_number INTEGER NOT NULL,
                course_id INTEGER NOT NULL,
                grade TEXT NOT NULL,
                FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
                FOREIGN KEY (course_id) REFERENCES courses (course_id)
            )
            ''')
        self.fill_db_with_sample_data()

    def fill_db_with_sample_data(self):
        # Sample data for students
        students = [
            (201001, 'Alice', 'Smith', '123 Park Ave'),
            (201002, 'Bob', 'Johnson', '456 Elm St'),
            (201003, 'Carol', 'Williams', '789 Oak St'),
            (201004, 'David', 'Brown', '101 Maple Dr'),
            (201005, 'Eve', 'Davis', '202 Pine Rd'),
            (201006, 'Frank', 'Miller', '303 Birch Blvd'),
            (201007, 'Grace', 'Wilson', '404 Cedar Ln'),
            (201008, 'Henry', 'Moore', '505 Spruce Ct'),
            (201009, 'Ivy', 'Taylor', '606 Aspen Way'),
            (201010, 'Jack', 'Anderson', '707 Walnut St')
        ]

        # Sample data for courses
        courses = [
            (101, 'Introduction to Computer Science', 'Dr. John Doe'),
            (102, 'Advanced Mathematics', 'Dr. Jane Smith'),
            (103, 'Physics for Engineers', 'Dr. Emily Johnson'),
            (104, 'Coding for Engineers', 'Dr. Emilia Johansson')
        ]

        # Insert students into the database
        self.execute_mass_mutation('INSERT INTO students VALUES (?,?,?,?)', students)

        # Insert courses into the database
        self.execute_mass_mutation('INSERT INTO courses VALUES (?,?,?)', courses)

        # Generate registrations (2 for each student)
        registrations = []
        for student in students:
            registered_courses = random.sample(courses, 2)  # Randomly pick 2 courses for each student
            for course in registered_courses:
                registrations.append((None, student[0], course[0]))

        # Insert registrations into the database
        self.execute_mass_mutation('INSERT INTO registrations (registration_id, matriculation_number, course_id) VALUES (?,?,?)', registrations)

        # Generate exam results (1 for each student)
        exam_results = []
        for student in students:
            course_id = random.choice(registrations)[2]  # Randomly pick one course from registrations
            grade = random.choice(['A', 'B', 'C', 'D', 'E', 'F'])  # Randomly assign a grade
            exam_results.append((None, student[0], course_id, grade))

        # Insert exam results into the database
        self.execute_mass_mutation('INSERT INTO exam_results (result_id, matriculation_number, course_id, grade) VALUES (?,?,?,?)', exam_results)
            

    def get_one(self, query, args=()):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.execute(query, args)
        result = cursor.fetchone()
        conn.close()
        return result
    
    def get_many(self, query):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()
        return result
    
    def execute_mutation(self, query, args = ()):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.execute(query, args)
        conn.commit()
        conn.close()

    def execute_mass_mutation(self, query, params = []):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.executemany(query, params)
        conn.commit()
        conn.close()   


In [ ]:

class ChatBot:
    def __init__(self):
        self.db = Db()
        self.prompts = []
        self.answers = []
        self.intent = None

    def chat(self, prompt):
        print("Bot: " + prompt)
        self.prompts.append(prompt)
        answer = input()
        print("You: "+ answer)
        self.answers.append(answer)
        self.process_input(answer)
        return answer
    
    def process_input(self, user_input = None):
        if user_input is None:
            user_input = self.answers[-1]
        tokens = word_tokenize(user_input)
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        return self.identify_intent(words)
    
    def identify_intent(self, processed_input):
        intent_keywords = {
            'register_exam': ['register', 'exam'],
            'deregister_exam': ['deregister', 'exam'],
            'change_address': ['change', 'address'],
            'change_surname': ['change', 'surname'],
            # Add more intents and associated keywords here for Query
        }

        for intent, keywords in intent_keywords.items():
            if all(keyword in processed_input for keyword in keywords):
                self.intent = intent
                return self.act()

        return 'unknown_intent'
    
    def act(self):
        if self.intent == 'unknown_intent':
            print("Sorry, I didn't understand.")
        elif self.intent == 'change_address':
            self.change_address()
        elif self.intent == 'change_surname':
            self.change_surname()
        elif self.intent == 'register_exam':
            self.register_exam()
        elif self.intent == 'deregister_exam':
            self.deregister_exam()
        # Add more intents and associated actions here
    
    def has_postcode(user_input):
        postcode_pattern = re.compile(r'\b\d{5}\b')
        return bool(re.search(postcode_pattern, user_input))
    
    def identify_yes_no_answer(self, input):
        intent_keywords = {
            'yes': ['true', 'yes','yeah', 'yep'],
            'no': ['wrong', 'false', 'no', 'not', 'nope', 'nah'],
            # Add more intents and associated keywords here
        }

        for intent, keywords in intent_keywords.items():
            if any(keyword in input for keyword in keywords):
                    return intent

        return 'unknown_intent'
    
    def change_surname(self):
        # Ask for student's matriculation number
        matriculation_number = self.chat("Please enter your matriculation number: ")
        
        
        # Check if student exists
        student=  self.db.get_one('SELECT * FROM students WHERE matriculation_number='+ matriculation_number)
        
        if student is None:
            print("Sorry, you are not registered as a student.")
        else:
            # Ask for new surname
            new_surname = self.chat("Please enter your new surname : ")
            # Confirm the updated surname
            answer = self.chat("Let me summarize once again: Your new surname is " + new_surname + "? ")
            processed_answer = self.identify_yes_no_answer(answer)
            
            while processed_answer == 'unknown_intent':
                answer = self.chat("Sorry, I didn't understand. Please answer with yes or no. ")
                processed_answer = self.identify_yes_no_answer(answer)
            
            if processed_answer == 'no':
                new_surname = self.chat("Please enter your correct new surname: ")
                # Update student's surname again
                self.db.execute_mutation('UPDATE students SET surname=? WHERE matriculation_number=?', (new_surname, matriculation_number))
            
            print("Your surname has been updated.")

            # Update student's surname
            self.db.execute_mutation('UPDATE students SET surname=? WHERE matriculation_number=?', (new_surname, matriculation_number))


    def query_exam_status(self, matriculation_number, course_id):
        registration = self.db.get_one('SELECT * FROM registrations WHERE matriculation_number=? AND course_id=?', (matriculation_number, course_id))
        if registration:
            return "Registered"
        else:
            return "Not Registered or Examination Completed"

    def query_exam_grade(self, matriculation_number, course_id):
        result = self.db.get_many('SELECT grade FROM exam_results WHERE matriculation_number=? AND course_id=?', (matriculation_number, course_id))
        if result:
            return result[0]
        else:
            return "No grade available or Examination not yet passed"
        
    def change_address(self):
        matriculation_number = self.chat("Please enter your matriculation number: ")
    #Check if student exists
        student=  self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number))
        if student is None:
            print("Sorry, you are not registered as a student.")
        else:
            #Ask for new address
            new_address = self.chat("Please enter your new address ")
            while not self.has_postcode(new_address):
                postal_code = self.chat("Can you give me your postal code please? ")
                new_address = new_address + " " + postal_code
            
            answer = self.chat("Let me summerize once again: You have moved out and your new address is " + new_address + "? ")
            processed_answer = self.identify_yes_no_answer(answer)
            while processed_answer == 'unknown_intent':
                answer = self.chat("Sorry, I didn't understand. Please answer with yes or no. ")
                processed_answer = self.identify_yes_no_answer(answer)
            if processed_answer == 'no':
                new_address = self.chat("Please enter your new address ")
                while not self.has_postcode(new_address):
                    postal_code = self.chat("Can you give me your postal code please? ")
                    new_address = new_address + " " + postal_code

            #Update student's address
            self.db.execute_mutation('UPDATE students SET address=? WHERE matriculation_number=?', (new_address, matriculation_number))
            print("Your address has been updated.")
    

    def register_exam(self):
        matriculation_number = self.chat("Please enter your matriculation number: ")

        # Check if student exists
        student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        if student is None:
            print("Sorry, you are not registered as a student.")
        else:
            while True:
                course_name = self.chat("Please enter the name of the course you want to register for: ")
                course = self.db.get_one('SELECT course_id FROM courses WHERE course_name = ?', (course_name,))

                if course is None:
                    print("Incorrect course name, please provide the correct one.")
                    continue

                answer = self.chat(f"Let me summarize once again: you want to register for the exam {course_name}? (yes/no)")
                processed_answer = self.identify_yes_no_answer(answer)

                while processed_answer == 'unknown_intent':
                    answer = self.chat("Sorry, I didn't understand. Please answer with yes or no.")
                    processed_answer = self.identify_yes_no_answer(answer)

                if processed_answer == 'yes':
                    # Check if already registered
                    registration = self.db.get_one('SELECT * FROM registrations WHERE matriculation_number = ? AND course_id = ?', (matriculation_number, course[0]))
                    if registration:
                        print(f"Student with matriculation number {matriculation_number} is already registered for course {course_name}.")
                        return

                    # Perform registration
                    self.db.execute_mutation('INSERT INTO registrations (matriculation_number, course_id) VALUES (?, ?)', (matriculation_number, course[0]))
                    print(f"Student with matriculation number {matriculation_number} registered for course {course_name}.")
                    return
                elif processed_answer == 'no':
                    break  # Restart the loop if the user says no


    def deregister_exam(self):
        matriculation_number = self.chat("Please enter your matriculation number: ")

        # Check if student exists
        student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        if student is None:
            print("Sorry, you are not registered as a student.")
        else:
            while True:
                course_name = self.chat("Please enter the name of the course you want to deregister from: ")
                course = self.db.get_one('SELECT course_id FROM courses WHERE course_name = ?', (course_name,))

                if course is None:
                    print("Incorrect course name, please provide the correct one.")
                    continue

                answer = self.chat(f"Let me summarize once again: you want to deregister from the exam {course_name}? (yes/no)")
                processed_answer = self.identify_yes_no_answer(answer)

                while processed_answer == 'unknown_intent':
                    answer = self.chat("Sorry, I didn't understand. Please answer with yes or no.")
                    processed_answer = self.identify_yes_no_answer(answer)

                if processed_answer == 'yes':
                    # Check if actually registered
                    registration = self.db.get_one('SELECT * FROM registrations WHERE matriculation_number = ? AND course_id = ?', (matriculation_number, course[0]))
                    if not registration:
                        print(f"Student with matriculation number {matriculation_number} is not registered for course {course_name}.")
                        return

                    # Perform deregistration
                    self.db.execute_mutation('DELETE FROM registrations WHERE matriculation_number = ? AND course_id = ?', (matriculation_number, course[0]))
                    print(f"Student with matriculation number {matriculation_number} deregistered from course {course_name}.")
                    return
                elif processed_answer == 'no':
                    break  # Exit the loop if the user says no




    

chatbot = ChatBot()

chatbot.chat("Hi, I am your chatbot. How can I help you?")
